function LevenshteinDistance(char s[1..m], char t[1..n]):

  // for all i and j, d[i,j] will hold the Levenshtein distance between
  // the first i characters of s and the first j characters of t
  declare int d[0..m, 0..n]
 
  set each element in d to zero
 
  // source prefixes can be transformed into empty string by
  // dropping all characters
  for i from 1 to m:
      d[i, 0] := i
 
  // target prefixes can be reached from empty source prefix
  // by inserting every character
  for j from 1 to n:
      d[0, j] := j
 
  for j from 1 to n:
      for i from 1 to m:
          if s[i] = t[j]:
            substitutionCost := 0
          else:
            substitutionCost := 1

          d[i, j] := minimum(d[i-1, j] + 1,                   // deletion
                             d[i, j-1] + 1,                   // insertion
                             d[i-1, j-1] + substitutionCost)  // substitution
 
  return d[m, n]

if s[i] = t[j] then 
  d[i, j] := d[i-1, j-1]
else if i > 0 and j > 0 and s[i] = t[j - 1] and s[i - 1] = t[j] then
  d[i, j] := minimum
             (
               d[i-2, j-2] + 1 // transpose
               d[i-1, j] + 1,  // deletion
               d[i, j-1] + 1,  // insertion
               d[i-1, j-1] + 1 // substitution
             )
else
  d[i, j] := minimum
             (
               d[i-1, j] + 1,  // deletion
               d[i, j-1] + 1,  // insertion
               d[i-1, j-1] + 1 // substitution
             )

In [3]:
def LD_swaps(s: str, t: str) -> int:
    '''
    '''
    
    d = [[0 for _ in range(len(t))] for _ in range(len(s))]
    
    for i in range(len(s)):
        d[i][0] = i
        
    for j in range(len(t)):
        d[0][1] = j
        
    for i in range(len(s)):
        for j in range(len(t)):
            
            if s[i] == t[j]:
                d[i][j] = d[i-1][j-1]
                
            elif i >0 and j >0 and s[i] == t[j-1] and s[i-1] == t[j]:
                d[i][j] = min([
                    d[i-2][j-2] + 1, # swap
                    d[i-1][j] + 1,   # deletion
                    d[i][j-1] + 1,   # substitution
                ])
            else:
                d[i][j] = min([
                    d[i-1][j] + 1,  # deletion
                    d[i][j-1] + 1,  # insertion
                    d[i-1][j-1] + 1,# substitution
                ])
    
    return d[len(s)-1][len(t)-1]

In [8]:
LD_swaps('ABCDEF', 'ABDCE')

2

In [49]:
def is_swap_up_to_dist(a: str, b: str, i: int, j: int, dist: int, d: list) -> list:
    '''
    Helper function to edit distance long swaps. Helps identify potential swaps
    and if two chars could be swapped then the entry to d is returned
    '''
    
    # first index not worth looking at
    if i == 0 or j == 0:
        return []
    
    # iterate up to our current index or the max swap distance
    iter_len = min([i, j, dist])
    
    # keep track of swaps we find
    swaps = []
    
    for k in range(1, iter_len + 1):
        
        # if it is a swap then keep track of it
        if a[i] == b[j - k] and a[i - k] == b[j]:
            swaps.append((i-k, j-k))
        
    return swaps

def edit_distance_long_swaps(a: str, b: str, dist: int = 0) -> int:
    '''
    Find the edit distance between two strings allowing for swaps up to a distance dist. 
    
    Example:
        a: 'ABCDE', b: 'ADCBE'
        
        edit distance with swap dist of 0: 2 (substitution of B for D and D for B)
        edit distance with swap dist of 1: 1 (swap of B and D)
        
    Limitations: if a position has 2 swaps, it will not be picked up. Example:
        a: 'CBEDA', b: 'ABCDE'
        
        A has been swapped with C then with E, but the final output will be edit distance of 3
        for all of the swaps
        
    Inputs:
        a:   (str) the first string
        b:   (str) the second string
        dist:(int) the swapping distance allowed. Default=0
        
    Outputs:
        (int) the minimum edit distance
    '''
    
    d = [[0 for _ in range(len(b))] for _ in range(len(a))]
    
    for i in range(len(a)):
        d[i][0] = i
        
    for j in range(len(b)):
        d[0][1] = j
        
    for i in range(len(a)):
        for j in range(len(b)):
            
            # look for swaps
            swaps = is_swap_up_to_dist(a, b, i, j, dist, d)
            
            if a[i] == b[j]:
                d[i][j] = d[i-1][j-1]
                
            elif len(swaps):
                
                # get all swaps possible 
                swap_values = [d[x][y] for x, y in swaps]
                                
                d[i][j] = min(swap_values + [
                    d[i-1][j] + 1,   # deletion
                    d[i][j-1] + 1,   # substitution
                ])
                
            else:
                d[i][j] = min([
                    d[i-1][j] + 1,  # deletion
                    d[i][j-1] + 1,  # insertion
                    d[i-1][j-1] + 1,# substitution
                ])
    
    return d[len(a)-1][len(b)-1]

In [50]:
edit_distance_long_swaps('ABCDEF', 'CBAFED', 2)

2